In [1]:
import mercury as mr
 
# add text widget
my_selection_stock = mr.Select(value = "NIFTY 50", choices = ["NIFTY 50", "Reliance Industries Limited", "Adani Enterprises Limited"], label = "Select stock")

mercury.Select

In [2]:
#print(f"Selected stock name is: {my_selection_stock.value}.")

In [3]:
import mercury as mr
 
# add text widget
my_selection_st = mr.Select(value = "Daily strategy", choices = ["Daily strategy", "Minute strategy"], label = "Select type")

mercury.Select

In [4]:
#print(f"Strategy will be executed as a {my_selection_st.value}.")

In [5]:
if my_selection_st.value == "Daily strategy":
    your_range = mr.Range(value=[2010, 2024], min=2010, max=2024, label="Your backtesting range", step=1)
    
    #access widget value in the code
    #print(f"Your strategy backtesting starts on 01-01-{your_range.value[0]} ends on 01-01-{your_range.value[1]}.")

mercury.Range

In [6]:
import yfinance as yf
import pandas as pd


if my_selection_stock.value == "NIFTY 50":
    symbol = "^NSEI"
elif my_selection_stock.value == "Adani Enterprises Limited":
    symbol = "ADANIENT.NS"
    
elif my_selection_stock.value == "Reliance Industries Limited":
    symbol = "RELIANCE.NS"

if my_selection_st.value == "Daily strategy":
    interval = "1d"
    start_date = f"{your_range.value[0]}-01-01"
    end_date = f"{your_range.value[1]}-01-01"
    df = yf.download(tickers=symbol, interval=interval, start=start_date, end=end_date)


elif my_selection_st.value == "Minute strategy":
    interval="1m"
    df = yf.download(tickers=symbol, interval=interval)

[*********************100%%**********************]  1 of 1 completed


In [8]:
from IPython.display import display, HTML

display(HTML(df.head(5).to_html()))
print("to")
display(HTML(df.tail(5).to_html()))

to


In [9]:
strategy = mr.Text(value="my strategy is the basic sma 200 vs sma 50.", label="describe your strategy here:", rows=4)

mercury.Text

In [10]:
get_ans = mr.Button(label="Get Backtested result", style="primary")
 
# after button clicked this will be true
# but it will be true only once!

mercury.Button

In [11]:
strategy_name="SMA_CrossOver"
strategy_code="""import backtrader as bt

class SMA_CrossOver(bt.Strategy):
    params = (('fast', 50), ('slow', 200),)

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    def __init__(self):
        self.sma1 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.fast)
        self.sma2 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.slow)
        self.crossover = bt.indicators.CrossOver(self.sma1, self.sma2)

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log('BUY CREATE, %.2f' % self.data.close[0])
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log('SELL CREATE, %.2f' % self.data.close[0])
            self.close()  # close long position
"""

In [12]:
if get_ans.clicked:
    from  langchain_community.document_loaders import PyMuPDFLoader
    loader = PyMuPDFLoader("main.pdf")
    documents = loader.load()
    from langchain_community.document_loaders import WebBaseLoader
    from langchain.vectorstores import FAISS
    from langchain.embeddings import OpenAIEmbeddings
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(documents)
    import os
    os.environ['OPENAI_API_KEY'] = ''
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))
    vector = FAISS.from_documents(documents, embeddings)
    retriever = vector.as_retriever()
    from langchain.tools.retriever import create_retriever_tool
    backtraderSearch = create_retriever_tool(
        retriever,
        "backtrader_search",
        "This tool contains the Documentation for the backtrader library which we are going to be using, when you are given a strategy, use this tool to see what indicators you are going to use, and how will you make a class using those indicators.",
    )
    from langchain.agents import AgentType, initialize_agent, load_tools
    from langchain_openai import ChatOpenAI, OpenAI
    tools = [backtraderSearch]
    from langchain_openai import ChatOpenAI

    model = ChatOpenAI(model="gpt-4")
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You will always be given a trading strategy as an input, the data will be provided to you, so you only hae to take help of the backtrader_search tool to see how will you make the class which I will later copy and paste and run it using backtrader api. Keep sure you are only making the class and nothing else. Also make the class such that when I run the cerebro engine I see what is hapening every day, that is after you are done with capturing the logic, make logs of the trade every day, example: class TestStrategy(bt.Strategy):

        def log(self, txt, dt=None):
            ''' Logging function fot this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

        def __init__(self):
            # Keep a reference to the "close" line in the data[0] dataseries
            self.dataclose = self.datas[0].close

        def next(self):
            # Simply log the closing price of the series from the reference
            self.log('Close, %.2f' % self.dataclose[0])

            if self.dataclose[0] < self.dataclose[-1]:
                # current close less than previous close

                if self.dataclose[-1] < self.dataclose[-2]:
                    # previous close less than the previous close

                    # BUY, BUY, BUY!!! (with all possible default parameters)
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.buy()
                    
                    see here in this very basic stategy we can see that how the buy/sell is printed, you have to do this for every class you make. Also make sure your position is always there, that is if you are at a buy position you can only sell, if you are at a sell position you can only buy next. Also make sure that array don't go out of range and also the function data.close is used properly.""",
            ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    llm_with_tools = model.bind_tools(tools)
    from langchain.agents.format_scratchpad.openai_tools import (
        format_to_openai_tool_messages,
    )
    from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    from langchain.agents import AgentExecutor

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    
    ans = list(agent_executor.stream({"input": strategy.value}))[0]['output']
    start = ans.find("python")
    if start != -1:
        # Find the first occurrence of triple quotes after the starting point
        end = ans.find("```", start)
        if end != -1:
            # Extract the content between the starting point and the triple quotes
            content = ans[start+len("python"):end]
            ans = content
        strategy_code = ans
        
    strategy_name = ans
    start = strategy_name.find("class ")
    if start != -1:
        # Find the first occurrence of triple quotes after the starting point
        end = strategy_name.find("(bt.Strategy)", start)
        if end != -1:
            # Extract the content between the starting point and the triple quotes
            content = strategy_name[start+len("class "):end]
            strategy_name = content
    
    #print(strategy_code)
    #print(strategy_name)


In [13]:
import backtrader as bt

In [14]:
exec(strategy_code)

In [15]:
data_feed = data_feed = bt.feeds.PandasData(dataname=df)

In [16]:
exec((f"""cerebro = bt.Cerebro()
cerebro.addstrategy({strategy_name})
cerebro.adddata(data_feed)
cerebro.broker.setcash(100000.0)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())"""))

Starting Portfolio Value: 100000.00
2012-03-14, BUY CREATE, 5463.90
2012-06-20, SELL CREATE, 5120.55
2012-07-27, BUY CREATE, 5099.85
2013-08-06, SELL CREATE, 5542.25
2013-11-01, BUY CREATE, 6307.20
2015-06-15, SELL CREATE, 8013.90
2016-05-31, BUY CREATE, 8160.10
2016-12-30, SELL CREATE, 8185.80
2017-02-17, BUY CREATE, 8821.70
2018-11-16, SELL CREATE, 10682.20
2019-03-15, BUY CREATE, 11426.85
2019-09-13, SELL CREATE, 11075.90
2019-11-11, BUY CREATE, 11913.45
2020-03-19, SELL CREATE, 8263.45
2020-08-21, BUY CREATE, 11371.60
2022-04-18, SELL CREATE, 17173.65
2022-09-09, BUY CREATE, 17833.35
2023-04-10, SELL CREATE, 17624.05
2023-06-01, BUY CREATE, 18487.75
Final Portfolio Value: 108458.65


In [17]:
from IPython.display import display
import matplotlib.pyplot as plt

# Assuming you have your strategy and data loaded into cerebro
# Replace `cerebro.plot()` with the following:


fig = cerebro.plot(style='bar', volume=False)[0][0]
fig.set_size_inches(12, 8)


# Use IPython's display to render the plot in the notebook
display(fig)

# Close the plot to avoid duplication in some environments
plt.close(fig)


<IPython.core.display.Javascript object>

<Figure size 1200x800 with 4 Axes>